# 使用低级别 TensorFlow API (TensorFlow Core) 编程

## 目录
- 使用 tf.Graph 和 tf.Session 管理 TensorFlow 程序

- 使用 tf.Session 运行 TensorFlow 操作。
- 构建自己的训练脚本。<br>

最好还是尽可能使用更高阶的 API 构建模型。
以下是 TensorFlow 低级API 为何很重要的原因：

- 如果您能够直接使用低阶 TensorFlow 操作，实验和调试都会更直接。
- 在使用更高阶的 API 时，能够理解其内部工作原理。

## 引入需要的包

In [3]:
import numpy as np
import tensorflow as tf

## Tensor Values （张量）
TensorFlow 中的核心数据单位是张量(**tensor**)。一个张量由一组阵列（**任意维数**）的原始值组成。张量的阶(**rank**)是它的维数，而它的形状(**shape**)是一个整数元组，指定了阵列每个维度的长度。<br>
TensorFlow 使用 numpy 阵列来表示张量值。<br>

以下是张量值的一些示例：

In [1]:
# 一个阶为0的tensor, shape是[]的标量
3. 
# 一个阶为1的tensor, shape是[3]的向量
[1., 2., 3.] 
# 一个阶为2的tensor, shape是[2， 3]的矩阵
[[1., 2., 3.], [4., 5., 6.]] 
# 一个阶为3的tensor, shape是[2, 1, 3]的矩阵
[[[1., 2., 3.]], [[7., 8., 9.]]]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

***请前往03_1_tensor, 03_2_variable***, 获得详细的tensor介绍

## Tensorflow Core
可以将 TensorFlow Core 程序看作由两个互相独立的部分组成：

- 构建计算图 (tf.Graph)。
- 运行计算图（使用 tf.Session）。

请看`3_graph_and_session`

## 层 （Layer）
可训练的模型必须修改图中的值，以便在输入相同值的情况下获得新的输出值。将可训练参数添加到图中的首选方法是层。<br>

层将变量和作用于它们的操作打包在一起。例如，全连接层会对每个输出对应的所有输入执行加权和，并应用激活函数（可选）。连接权重和偏差由层对象管理。

tensorflow 低级api定义在tf.layers与tf.nn.layers中

- tf.nn
提供神经网络相关操作的支持，包括卷积操作（conv）、池化操作（pooling）、归一化、loss、分类操作、embedding、RNN、Evaluation。

- tf.layers
主要提供的高层的神经网络，主要和卷积相关的，是对tf.nn的进一步封装，tf.nn会更底层一些。

### 创建层

In [43]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

层会检查其输入数据，以确定其内部变量的大小。因此，我们必须在这里设置 x 占位符的形状，以便层构建正确大小的权重矩阵。

我们现在已经定义了输出值 y 的计算，在我们运行计算之前，还需要处理一个细节。
### 初始化层

In [44]:
init = tf.global_variables_initializer()
sess.run(init)

global_variables_initializer 仅会初始化创建初始化程序时图中就存在的变量。因此应该在构建图表的最后一步添加初始化程序。
### 执行层
我们现在已经完成了层的初始化，可以像处理任何其他张量一样评估 linear_model 的输出张量了。例如，下面的代码：

In [45]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[ 4.778865]
 [10.312775]]


## 训练
已经了解 TensorFlow 核心部分的基础知识了，我们来手动训练一个小型回归模型吧。
### 定义数据
我们首先来定义一些输入值 `x`，以及每个输入值的预期输出值 `y_true`：

In [46]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

### 定义模型
接下来，建立一个简单的线性模型，其输出值只有 1 个：

In [47]:
linear_model = tf.layers.Dense(units=1)

# 进行预测、还没有训练，所以效果不是很好
y_pred = linear_model(x)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[0.5243298]
 [1.0486596]
 [1.5729893]
 [2.0973191]]


### 损失
要优化模型，首先需要定义损失。我们将使用均方误差，这是回归问题的标准损失。

虽然可以使用较低级别的数学运算手动定义，但 tf.losses 模块提供了一系列常用的损失函数。可以使用它来计算均方误差，具体操作如下所示：

In [48]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

10.805211


### 训练
TensorFlow 提供了执行标准优化算法的优化器。这些优化器被实现为 tf.train.Optimizer 的子类。它们会逐渐改变每个变量，以便将损失最小化。最简单的优化算法是梯度下降法，由 tf.train.GradientDescentOptimizer 实现。它会根据损失相对于变量的导数大小来修改各个变量。例如：

In [49]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

该代码构建了优化所需的所有图组件，并返回一个训练指令。该训练指令在运行时会更新图中的变量。您可以按以下方式运行该指令：

In [50]:
for i in range(10):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

10.805211
7.590361
5.35909
3.8103056
2.7350888
1.9884733
1.4698704
1.1094842
0.85888433
0.684466


## 实践
请将上面的程序写成一个完整的程序